In [1]:
import pandas as pd
import numpy as np
from typing import *
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    # print(last_hidden.shape)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(r"Embedding_Model/gte_large")
model = AutoModel.from_pretrained(r"Embedding_Model/gte_large")

In [ ]:
# val1=tokenizer.save_pretrained(r"Embedding_Model/gte_large")
# val2= model.save_pretrained(r"Embedding_Model/gte_large/")

(('Embedding_Model/gte_large\\tokenizer_config.json',
  'Embedding_Model/gte_large\\special_tokens_map.json',
  'Embedding_Model/gte_large\\vocab.txt',
  'Embedding_Model/gte_large\\added_tokens.json',
  'Embedding_Model/gte_large\\tokenizer.json'),
 None)

In [5]:
data_input = pd.read_csv(r"Generated_Result/three_output1.csv", sep=";")

In [7]:
stupid: Dict[str, List] = {}
for col in ["Project", "Risk", "Stakeholder"]:
    temp = data_input.loc[:, [col]]
    blyat = []
    for tep_val in temp.values.tolist():
        if tep_val[0] != tep_val[0]: continue
        tep_val = tep_val[0].split("| ")
        blyat = [*blyat, *tep_val]
    stupid[col] = blyat

# stupid_copy = {col: [val for val in data_input[col].str.split("|", expand=True).stack().dropna()]\
#            for col in ["Project", "Risk", "Stakeholder"]}

In [12]:
temp_project = [stupid["Risk"][val: val+5] for val in range(0, len(stupid["Risk"]), 5)]

In [15]:
for col in ["Stakeholder", "Project"]:
    temp_project = [stupid[col][val: val+3] for val in range(0, len(stupid[col]), 3)]
    output_project = []
    for ele in temp_project:
        # Tokenize the input texts
        batch_dict = tokenizer(ele, max_length=512, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        output_project.append(embeddings)
        torch.save(torch.cat(output_project, axis=0), f"Embedding_Model/gte_large_embedding/three_ouput1_{col}.pt")

In [14]:
# beside stakeholder, rest of them shall be reloaded again

torch.save(torch.cat(output_project, axis=0), "Embedding_Model/gte_large_embedding/three_ouput1_risk.pt")
# print(len(stupid["Project"]), torch.cat(output_project, axis=0).shape)

In [16]:
# temp_project=0
# embeddings = 0
# stupid["Project"]=0
import gc
gc.collect()

0

In [13]:
full_emd, atten_mask = outputs.last_hidden_state, batch_dict['attention_mask']

In [14]:
print(full_emd.shape, atten_mask.shape, embeddings.shape)

torch.Size([4, 6, 768]) torch.Size([4, 6]) torch.Size([4, 768])


In [13]:
# (Optionally) normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:1] @ embeddings[1:].T) * 100
print(scores.tolist())

[[69.65817260742188, 88.03556060791016, 68.79690551757812]]


In [1]:
import pandas as pd

In [7]:
d1 = pd.read_csv(r"Generated_Result/1k_fp.csv", sep=';')
d1.to_excel(r"Generated_Result/new_split_quota.xlsx", index=False)